In [1]:
addprocs(4)
import DifferentialEquations
import ParameterizedFunctions
@everywhere using DifferentialEquations
@everywhere using ParameterizedFunctions
using Plots
using LsqFit
using ProgressMeter
using PmapProgressMeter
gr();

In [2]:
@everywhere de_system = @ode_def_noinvjac Sips2 begin
    dpx = 1
    dpy = 2*pi*sin(2*pi*t/5)
    dkx = kvx
    dky = kvy
    # We add a very small float value to the denominator so we never divide by zero
    dkvx = (px - kx)/sqrt((px - kx)^2 + (py - ky)^2+0.00000000001)/m
    dkvy = (py - ky)/sqrt((px - kx)^2 + (py - ky)^2+0.00000000001)/m
end m=>1.0

In [12]:
time_max = 1000000
u0 = [0.0;0.0;0.0;0.0;0.0;0.0]
de_system.m = 106
tspan = (0.0,1.0 * time_max)
prob = ODEProblem(de_system, u0, tspan)

@time sol = solve(prob, Trapezoid(), dt=0.5, maxiters=1e10)
#@time sol = solve(prob, Euler(), dt=0.5, maxiters=1e10);
#@time sol = solve(prob, Vern9());

 57.398141 seconds (138.18 M allocations: 11.256 GB, 55.70% gc time)


In [ ]:
@time T = linspace(0,50000,10000)
@time R = hcat(map(sol, T)...)
px, py, kx, ky = R[1,:], R[2,:], R[3,:], R[4,:]
#plot(px, py)
plot(kx, ky)
xaxis!("x")
yaxis!("y")
title!("Long term solution behavior")

In [ ]:
@time wave_function = zeros(Int(time_max))
@showprogress 1 "Computing..." for t in linspace(0, time_max, time_max*2)
    (px, py, x, y, vx, vy) = sol(t)
    wave_function[min(round(Int, x)+1,end)] = y
end
#wave_function -= mean(wave_function)
plot(1:10:min(length(wave_function),20000),wave_function[1:10:20000])
xaxis!("x")
yaxis!("y")
title!("De-parameterized solution")

In [ ]:
@time T = linspace(1300,1400,500)
@time R = hcat(map(sol, T)...)
px, py, kx, ky = R[1,:], R[2,:], R[3,:], R[4,:]
anim_dt = 100
@gif for i in 1:length(px)
    @printf("%d\r", i)
    flush(STDOUT)
    plot(px[1:i], py[1:i])
    plot!(kx[1:i], ky[1:i])
end every 1

In [ ]:
time_max = 1e6
        
# Now convert to frequency space using a Fast Fourier Transform
# Lots of random constants floating around to make the frequency mappings work.
dT = 1 # This is 1 because we are working in index space
yf = 2.0/(time_max/dT) * abs(rfft(wave_function))
xf = linspace(0, 0.5/dT, div(time_max/dT,2))
pks= zeros(length(xf))
#pks[indmax(yf[10:div(indmax(yf),2)])+9] = 0.2
#1/xf[indmax(yf[10:div(indmax(yf),2)])+9]
#maxidx=round(Int, sum(x -> (if yf[x]>0.04 yf[x] else 0 end)*x,9:200)/sum(x -> (if yf[x]>0.04 yf[x] else 0 end), 9:200))
maxidx=indmax(yf[10:div(indmax(yf[10:end])+9,2)])+9
#maxidx = findfirst(x -> x > 0.05, yf[10:end])+9
pks[maxidx]=0.1
1/xf[maxidx]

In [ ]:
plot_crop = 10000
plot(xf[9:1:min(plot_crop,end)], yf[9:1:min(plot_crop,end)])
plot!(xf[9:1:min(plot_crop,end)], pks[9:1:min(plot_crop,end)])
xaxis!("Frequency")
yaxis!("Amplitude")

In [13]:
@time T = linspace(0, 5000, 50000)
@time R = hcat(map(sol, T)...)
anim_dt = 100
@gif for i in 1:anim_dt:50000
    @printf("%d\r", i)
    flush(STDOUT)
    px, py, kx, ky = R[1,1:i], R[2,1:i], R[3,1:i], R[4,1:i]
    plot(px, py)
    plot!(kx, ky)
end every 1

  0.000003 seconds (5 allocations: 208 bytes)
  0.082376 seconds (496.95 k allocations: 22.079 MB, 63.11% gc time)


INFO: Saved animation to /home/pythonnut/Documents/prog/sipz/tmp.gif


Plots.AnimatedGif("/home/pythonnut/Documents/prog/sipz/tmp.gif")

In [ ]:
function mass_to_period(kozai_masses)
    # Euler's Method time resolution
    pmap(
        function (kozai_mass)
            time_max=1000000
            # Set initial conditions
            u0 = [0.0; 0.0; 0.0; 0.0; 0.0; 0.0]
            de_system.m = kozai_mass
            tspan = (0.0, 1.0*time_max)

            # Solve using Hairer's 8/5/3 adaption of the Dormand-Prince 8 Runge-Kutta method. 
            # (7th order interpolant)
            # Oddly, we need a super high-order interpolant, otherwise our
            # cumulative errors get out of hand. Vern9 was the only method that 
            # could solve this system with high accuracy in reaonable time
            prob = ODEProblem(de_system, u0, tspan)
            sol = solve(prob, Vern9())

            # Now we "flatten" the parametric position array [[..., x, y, ...]...]
            # into a index vs. y array [y1, y2 ...]. We do this so we can later
            # pass the array to the DFT, which takes a glat array as input.
            # NOTE: We don't perform any scaling. index <==> x
            # NOTE: This only makes sense when the parametric curve passes
            # the vertical line test, which, in many cases, it does not.
            wave_function = zeros(time_max)
            for t in linspace(0, time_max, time_max * 2)
                (px, py, x, y, vx, vy) = sol(t)
                wave_function[min(round(Int, x)+1,end)] = y
            end

            # Make sure the trajectory is centered around zero.
            # Otherwise, the FFT will produce a mess trying to build
            # a constant out of sines. 
            #wave_function -= mean(wave_function)

            # Now convert to frequency space using a Fast Fourier Transform
            # Lots of random constants floating around to make the frequency mappings work,
            # since the real-valued FFT only returns an array of half size
            dT = 1 # This is 1 because we are working in index space
            yf = 2.0/(time_max/dT) * abs(rfft(wave_function))
            xf = linspace(0, 0.5/dT, div(time_max/dT,2))
            #1/xf[findmax(yf[10:div(findmax(yf)[2],2)])[2]]

            # A heuristic to extract the "low" frequency oscillation. Essentially, 
            # we find the strongest signal, which is the  "high" frequency 
            # oscillation and look for a peak at less than half that frequency.
            try
                fast_idx = indmax(yf[10:end])+9
                #maxidx=indmax(yf[10:div(indmax(yf[10:end])+9,2)])+9
                #maxidx=floor(Int, sum(x -> (if yf[x]>0.05 yf[x] else 0 end)*x,9:200)/sum(x -> (if yf[x]>0.05 yf[x] else 0 end), 9:200))
                #maxidx=findfirst(x -> x > 0.04, yf[10:end])+9
                fast_period = 1/xf[fast_idx]
                slow_idx = indmax(yf[10:div(fast_idx, 2)])+9
                slow_period = 1/xf[slow_idx]
                
                (slow_period, fast_period)
            catch
                0
            end
        end, 
        Progress(length(kozai_masses)), kozai_masses)
end;

In [ ]:
X = linspace(50, 450, 50)
periods = mass_to_period(X)

In [ ]:
# was 111110.88888888889
periods[107] = (periods[106] + periods[108])/2

In [ ]:
plot(X, periods)
xaxis!("Mass")
yaxis!("Long-range period")
title!("Period vs. Mass")

In [ ]:
quad(x, p) = p[1] + p[2].*x + p[3].*x.^2
fit = curve_fit(quad, X, periods, [0.0,0.0,0.0])
plot(X, periods)
plot!(X, map(x -> quad(x, fit.param), X))
xaxis!("Mass")
yaxis!("Long-range period")
title!("Period vs. Mass")

In [ ]:
(fit.param, estimate_errors(fit,0.95))

In [ ]:
for (t, p) in zip(X, periods)
    @printf("%f: %f\n", t, p)
end